# 00 - Problem (case study)

Data Description:
We have received data from the FIFA21-game. 
stating the personal information of the players (ID	Name	Age	Nationality	Club), physical characteristics (Height	Weight	foot), contractual information (Joined	Loan Date End	Value	Wage	Release Clause	Contract), sporting/game characteristics (BP	Position	Team & Contract		Growth		Attacking	Crossing	Finishing	Heading Accuracy	Short Passing	Volleys	Skill	Dribbling	Curve	FK Accuracy	Long Passing	Ball Control	Movement	Acceleration	Sprint Speed	Agility	Reactions	Balance	Power	Shot Power	Jumping	Stamina	Strength	Long Shots	Mentality	Aggression	Interceptions	Positioning	Vision	Penalties	Composure	Defending	Marking	Standing Tackle	Sliding Tackle	Goalkeeping	GK Diving	GK Handling	GK Kicking	GK Positioning	GK Reflexes	Total Stats	Base Stats	W/F	SM	A/W	D/W	IR	PAC	SHO	PAS	DRI	DEF	PHY	Hits	LS	ST	RS	LW	LF	CF	RF	RW	LAM	CAM	RAM	LM	LCM	CM	RCM	RM	LWB	LDM	CDM	RDM	RWB	LB	LCB	CB	RCB	RB	GK	OVA)

The goal of the data analysis is to be able, based on the information above to predict the overall average that ranks all the real football players for the FIFA21-game. We will as well validate the data with a new dataset.

# 01 - Import libraries

In [1]:
import pandas as pd  # keywords are highlighted in green, other strings in red, etc.
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#for plots
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#for regression
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler # do not use the function Normalise() - it does something entirely different
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

#categorical Variables
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

#for stats
import scipy.stats as stats

# 02 - Getting Data

In [2]:
fifa = pd.read_csv('./fifa21_train.csv')
fifa.shape
fifa.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,184383,A. Pasche,26,Switzerland,FC Lausanne-Sport,CM,CM CDM,FC Lausanne-Sport 2015 ~ 2020,"5'9""",161lbs,...,59+1,59+1,59+1,58+1,54+1,54+1,54+1,58+1,15+1,64
1,188044,Alan Carvalho,30,China PR,Beijing Sinobo Guoan FC,ST,ST LW LM,"Beijing Sinobo Guoan FC Dec 31, 2020 On Loan","6'0""",159lbs,...,53+2,53+2,57+2,53+2,48+2,48+2,48+2,53+2,18+2,77
2,184431,S. Giovinco,33,Italy,Al Hilal,CAM,CAM CF,Al Hilal 2019 ~ 2022,"5'4""",134lbs,...,56+2,56+2,59+2,53+2,41+2,41+2,41+2,53+2,12+2,80
3,233796,J. Evans,22,Wales,Swansea City,CDM,CDM CM,Swansea City 2016 ~ 2021,"5'10""",152lbs,...,58+2,58+2,56+2,57+2,58+2,58+2,58+2,57+2,14+2,59
4,234799,Y. Demoncy,23,France,US Orléans Loiret Football,CDM,CDM CM,US Orléans Loiret Football 2018 ~ 2021,"5'11""",150lbs,...,64+2,64+2,64+2,63+2,61+2,61+2,61+2,63+2,15+2,65


# 03 - Cleaning/Wrangling - Standardize header names

Here we build a function "clean_data" that summarizes all the standardization and cleaning steps we had in the dataframe: "fifa".

In [3]:
def clean_data(df):
    
    #change column names
    df=df.rename(columns={
                            'FK Accuracy':'Free kick Accuracy',
                            'GK Diving':'Goalkeeper Diving',
                            'GK Handling':'Goalkeeper Handling',
                            'GK Kicking':'Goalkeeper Kicking',
                            'GK Positioning':'Goalkeeper Position',
                            'GK Reflexes':'Goalkeeper Reflexes',
                            'W/F':'Weak foot',
                            'SM':'Skill Moves',
                            'A/W':'Away Win',
                            'D/W':'Defensive Workrate',
                            'IR':'International Reputation',
                            'PAC':'Pace',
                            'SHO':'Shooting',
                            'PAS':'Passing',
                            'DRI':'Dribbling',
                            'DEF':'Defending',
                            'PHY':'Physicality',
                            'LS':'LS_Left Striker',
                            'ST':'ST_Striker',
                            'RS':'RS_Right Striker',
                            'LW':'LW_Left Winger',
                            'LF':'LF_Left Forward',
                            'CF':'CF_Center Forward',
                            'RF':'RF_Right Forward',
                            'RW':'RW_Right Winger',
                            'LAM':'LAM_Left Attacking Midfielder',
                            'CAM':'CAM_Center Attacking Midfielder',
                            'RAM':'RAM_Right Attacking Midfielder',
                            'LM':'LM_Left Midfielder',
                            'LCM':'LCM_Left Center Midfielder',
                            'CM':'CM_Center Midfielder',
                            'RCM':'RCM_Right Center Midfielder',
                            'RM':'RM_Right Midfielder',
                            'LWB':'LWB_Left Wing Back',
                            'LDM':'LDM_Left Defensive Midfielder',
                            'CDM':'CDM_Center Defensive Midfielder',
                            'RDM':'RDM_Right Defensive Midfielder',
                            'RWB':'RWB_Right Wing Back',
                            'LB':'LB_Left Back',
                            'LCB':'LCB_Left Center Back',
                            'CB':'CB_Center Back',
                            'RCB':'RCB_Right Center Back',
                            'RB':'RB_Right Back',
                            'GK':'GK_Goalkeeper',
                            'OVA':'OVA_Overall Average'
                            })
    
    # standardize headers
    df.columns = [e.lower().replace(' ', '_') for e in df.columns]
       
    # string to numeric
    df['value'] = df['value'].str.replace('€','')
    df['wage'] = df['wage'].str.replace('€','')
    df['release_clause'] = df['release_clause'].str.replace('€','')

    df['value'] = df['value'].str.replace('K', '000')
    df['wage'] = df['wage'].str.replace('K', '000')
    df['release_clause'] = df['release_clause'].str.replace('K', '000')

    df['value'] = df['value'].str.replace('M', '000000')
    df['wage'] = df['wage'].str.replace('M', '000000')
    df['release_clause'] = df['release_clause'].str.replace('M', '000000')

    df['value'] = pd.to_numeric(df['value'], errors='coerce')
    df['wage'] = pd.to_numeric(df['wage'], errors='coerce')
    df['release_clause'] = pd.to_numeric(df['release_clause'], errors='coerce')
        
    # make this 
    df['weak_foot'] = df['weak_foot'].astype(str).str[0]
    df['skill_moves'] = df['skill_moves'].astype(str).str[0]
    df['international_reputation'] = df['international_reputation'].astype(str).str[0]

    df['weak_foot'] = pd.to_numeric(df['weak_foot'], errors='coerce')
    df['skill_moves'] = pd.to_numeric(df['skill_moves'], errors='coerce')
    df['international_reputation'] = pd.to_numeric(df['international_reputation'], errors='coerce')
    
    #convert height to cm and numeric
    df['height'] = df['height'].str.replace('"','')
    conversions = [30.48, 2.54]
    df['height'] = df['height'].str.split("'").apply(pd.Series).astype(int).dot(conversions)

    #convert weight to kg and numeric
    df['weight'] = df['weight'].str.replace('lbs','')
    df['weight'] = pd.to_numeric(df['weight'], errors='coerce')*0.453592
    
    ### lets round the values to 2 decimals
    df['weight'] = round(df['weight'], 2) 
    df['height'] = round(df['height'], 2) 
    
    #changing header cm and kg
    df = df.rename(columns={'weight': 'weight_kg', 'height': 'height_cm'})
        
    ### date/time format
    df['joined'] = pd.to_datetime(df['joined'], errors='coerce')
    df['loan_date_end'] = pd.to_datetime(df['loan_date_end'], errors='coerce')
    
    #convert to numeric
    df['hits'] = pd.to_numeric(df['hits'], errors='coerce')
    
   # dropping useless columns
    df = df.drop(columns=['loan_date_end'], axis=1)
    df = df.drop(columns=['contract'], axis=1)
    df = df.drop(columns=['joined'], axis=1)
    df = df.drop(columns=['team_&_contract'], axis=1)
    df = df.drop(columns=['name'], axis=1)
    df = df.drop(columns=['nationality'], axis=1)
    df = df.drop(columns=['id'], axis=1)
    df = df.drop(columns=['club'], axis=1)
    df = df.drop(columns=['position'], axis=1)
    df = df.drop(columns=['ls_left_striker',
                             'st_striker',
                             'rs_right_striker',
                             'lw_left_winger',
                             'lf_left_forward',
                             'cf_center_forward',
                             'rf_right_forward',
                             'rw_right_winger',
                             'lam_left_attacking_midfielder',
                             'cam_center_attacking_midfielder',
                             'ram_right_attacking_midfielder',
                             'lm_left_midfielder',
                             'lcm_left_center_midfielder',
                             'cm_center_midfielder',
                             'rcm_right_center_midfielder',
                             'rm_right_midfielder',
                             'lwb_left_wing_back',
                             'ldm_left_defensive_midfielder',
                             'cdm_center_defensive_midfielder',
                             'rdm_right_defensive_midfielder',
                             'rwb_right_wing_back',
                             'lb_left_back',
                             'lcb_left_center_back',
                             'cb_center_back',
                             'rcb_right_center_back',
                             'rb_right_back',
                             'gk_goalkeeper'], axis=1)
# dropping nan values
    df = df[df['volleys'].isna()==False]
    df = df[df['composure'].isna()==False]
    df = df[df['away_win'].isna()==False]
    df = df[df['defensive_workrate'].isna()==False]
    df = df[df['hits'].isna()==False]
    
    return df

In [4]:
#check the columns and the data types, with the first column.
def fifa_colandtype(x):
    fifa_col = pd.DataFrame(x.iloc[0 , :])
    fifa_col['type'] = x.dtypes
    fifa_col = fifa_col.reset_index()
    fifa_col.columns = ['header_name', 'row 0', 'type']
    pd.set_option('display.max_rows', None)
    display(fifa_col)
    return

In [5]:
# #Check columns and types
# fifa_colandtype(fifa)

In [6]:
fifa = clean_data(fifa)

# 04 - Cleaning/Wrangling - Check and deal with NaN values.

In [7]:
#Check for duplicate rows in the data and remove if any.
sum(fifa.duplicated()) #check how many rows are duplicated.
fifa = fifa.drop_duplicates()
fifa.shape

(11380, 65)

In [8]:
def get_percentnan(x):
    fifa_percentnan = pd.DataFrame(round(x.isna().sum()/len(x),4)*100)
    fifa_percentnan = fifa_percentnan.reset_index()
    fifa_percentnan.columns = ['header_name', 'percent_nulls']
    fifa_percentnan = fifa_percentnan[(fifa_percentnan['percent_nulls'] > 0)]
    display(fifa_percentnan)
    return

In [9]:
get_percentnan(fifa) # --> all the NaN-Values are dropped !

,header_name,percent_nulls


In [10]:
fifa.shape

(11380, 65)

In [11]:
fifa.head()

,age,bp,height_cm,weight_kg,foot,growth,value,wage,release_clause,attacking,...,defensive_workrate,international_reputation,pace,shooting,passing,dribbling,defending,physicality,hits,ova_overall_average
0,26,CM,175.26,73.03,Right,1,525000.0,4000,801000.0,258,...,Medium,1,69,51,63,63,51,60,3.0,64
1,30,ST,182.88,72.12,Right,0,8.5,23000,0.0,365,...,Low,2,83,75,68,82,33,71,44.0,77
2,33,CAM,162.56,60.78,Right,0,9000000.0,49000,15.3,336,...,Medium,2,80,77,78,86,27,56,73.0,80
3,22,CDM,177.80,68.95,Right,13,275000.0,4000,694000.0,242,...,Medium,1,57,44,54,57,57,60,7.0,59
4,23,CDM,180.34,68.04,Right,8,725000.0,2000,1.4,249,...,Medium,1,66,44,60,64,60,66,4.0,65


In [12]:
#check the columns and the data types.
fifa_colandtype(fifa)

,header_name,row 0,type
0,age,26,int64
1,bp,CM,object
2,height_cm,175.26,float64
3,weight_kg,73.03,float64
4,foot,Right,object
5,growth,1,int64
6,value,525000.0,float64
7,wage,4000,int64
8,release_clause,801000.0,float64
9,attacking,258,int64


# 05 - EDA (Exploratory Data Analysis)

In [13]:
#DataFrame info
info_data = fifa.info()
info_data

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11380 entries, 0 to 11700
Data columns (total 65 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       11380 non-null  int64  
 1   bp                        11380 non-null  object 
 2   height_cm                 11380 non-null  float64
 3   weight_kg                 11380 non-null  float64
 4   foot                      11380 non-null  object 
 5   growth                    11380 non-null  int64  
 6   value                     11380 non-null  float64
 7   wage                      11380 non-null  int64  
 8   release_clause            11380 non-null  float64
 9   attacking                 11380 non-null  int64  
 10  crossing                  11380 non-null  int64  
 11  finishing                 11380 non-null  int64  
 12  heading_accuracy          11380 non-null  int64  
 13  short_passing             11380 non-null  int64  
 14  volley

In [14]:
# describing the numerical columns
summary_numerical = fifa.describe().T 

# additionnaly we add the range and the iqr
summary_numerical['iqr'] = summary_numerical['75%']-summary_numerical['25%']
summary_numerical['range'] = summary_numerical['max']-summary_numerical['min']
summary_numerical

#round up the values to 2 decimals (round() would round to 0 decimals) --> with function
def roundforme(x):
    return round(x,2)

#to round the whole table..
for col in summary_numerical.columns:
    summary_numerical[col] = summary_numerical[col].apply(roundforme)
summary_numerical

,count,mean,std,min,25%,50%,75%,max,iqr,range
age,11380.0,25.17,4.87,16.00,21.00,25.00,29.00,4.300000e+01,8.00,2.700000e+01
height_cm,11380.0,181.23,6.91,154.94,175.26,180.34,185.42,2.057400e+02,10.16,5.080000e+01
weight_kg,11380.0,75.10,7.16,53.07,69.85,74.84,79.83,1.102200e+02,9.98,5.715000e+01
growth,11380.0,5.58,5.82,0.00,0.00,4.00,10.00,2.600000e+01,10.00,2.600000e+01
value,11380.0,1111218.29,3860921.50,0.00,4.20,275000.00,650000.00,9.000000e+07,649995.80,9.000000e+07
wage,11380.0,10193.08,20907.24,0.00,1000.00,3000.00,10000.00,5.600000e+05,9000.00,5.600000e+05
release_clause,11380.0,671242.60,3837542.29,0.00,1.80,10.30,481000.00,1.610000e+08,480998.20,1.610000e+08
attacking,11380.0,257.89,72.22,42.00,231.00,271.00,305.00,4.370000e+02,74.00,3.950000e+02
crossing,11380.0,51.45,17.85,6.00,40.00,56.00,65.00,9.400000e+01,25.00,8.800000e+01
finishing,11380.0,47.88,19.38,3.00,33.00,52.00,63.00,9.500000e+01,30.00,9.200000e+01


# 06 - Processing Data - X-y split

In [15]:
y = fifa['ova_overall_average']
X = fifa.drop(['ova_overall_average'], axis=1)
X.head()
X.shape # one column is dropped, ok.

(11380, 64)

In [16]:
X_num = X.select_dtypes(np.number)
X_cat = X.select_dtypes(object)

In [17]:
X_num.shape

(11380, 60)

In [18]:
X_cat.shape

(11380, 4)

# 06 - Processing Data - Normalize numerical data

Normalization or Min-Max Scaling is used to transform features to be on a similar scale. 

The new point is calculated as:
X_new = (X - X_min)/(X_max - X_min)

This scales the range to [0, 1] or sometimes [-1, 1]. 
Geometrically speaking, transformation squishes the n-dimensional data into an n-dimensional 
unit hypercube. Normalization is useful when there are no outliers as it cannot cope up with 
them. Usually, we would scale age and not incomes because only a few people have high incomes 
but the age is close to uniform.

In [19]:
# This is the code for the MIN-MAX-Normalizing, in this lab I chose to do the Standard Normalizing, 
# because it takes into account more the outliers.

# Normalizing data: make data range from 0 - 1, instead of from min to max
transformer = MinMaxScaler().fit(X_num)
X_num_normalized = transformer.transform(X_num)
print(X_num_normalized.shape)

X_num_normalized=pd.DataFrame(X_num_normalized, columns=X_num.columns)
#X_num_normalized

(11380, 60)


# 06 - Processing Data - One Hot/Label Encoding for categorical data

In [20]:
X_cat.head()

,bp,foot,away_win,defensive_workrate
0,CM,Right,High,Medium
1,ST,Right,High,Low
2,CAM,Right,High,Medium
3,CDM,Right,Medium,Medium
4,CDM,Right,Low,Medium


In [21]:
#one hot encoding is a way to turn categorical variables into multiple numerical columns
encoder = OneHotEncoder(drop='first').fit(X_cat) # the first one is the most frequent one.
#print(encoder.categories_)
encoded = encoder.transform(X_cat).toarray()

cols = encoder.get_feature_names(input_features=X_cat.columns)
cols

onehot_encoded = pd.DataFrame(encoded, columns=cols)
onehot_encoded.head()
onehot_encoded.shape
onehot_encoded.columns

Index(['bp_CB', 'bp_CDM', 'bp_CF', 'bp_CM', 'bp_GK', 'bp_LB', 'bp_LM', 'bp_LW',
       'bp_LWB', 'bp_RB', 'bp_RM', 'bp_RW', 'bp_RWB', 'bp_ST', 'foot_Right',
       'away_win_Low', 'away_win_Medium', 'defensive_workrate_Low',
       'defensive_workrate_Medium'],
      dtype='object')

#### Concat DataFrames

In [22]:
X = pd.concat([X_num_normalized, onehot_encoded], axis=1)

# 07 - Linear Regression - Correlation Analysis

In [23]:
y = y.reset_index(drop=True)
X = X.reset_index(drop=True)

### Correlations Index:

- 0.9 and 1.0 very highly correlated
- 0.7 and 0.9 highly correlated
- 0.5 and 0.7 moderately correlated
- 0.3 and 0.5 low correlation
- less than 0.3 little if any (linear) correlation. 

In [24]:
def getcollinearity(filtered_matrix, column_to_check):
    collinearity = filtered_matrix[(filtered_matrix[column_to_check] >= 0)]
    collinearity = collinearity[column_to_check]
    return collinearity

In [25]:
#correlation
Collinearity = pd.concat((y, X), axis=1)
corr_matrix_cat = round((Collinearity.corr()),2)
corr_matrix_cat

filtered_matrix = corr_matrix_cat[((corr_matrix_cat >= .0) | (corr_matrix_cat <= -.0))]

# (1) We filter only the values that are highly correlated
filtered_matrix_high = corr_matrix_cat[((corr_matrix_cat >= .7) | (corr_matrix_cat <= -.7))]
getcollinearity(filtered_matrix_high,'ova_overall_average')

ova_overall_average    1.00
reactions              0.87
composure              0.70
base_stats             0.84
passing                0.70
Name: ova_overall_average, dtype: float64

In [26]:
# (2) We filter only the values that are moderatly correlated
filtered_matrix_moderate = corr_matrix_cat[((corr_matrix_cat <= .7) & (corr_matrix_cat >= .5) | (corr_matrix_cat >= -.7) & (corr_matrix_cat <= -.5))]
getcollinearity(filtered_matrix_moderate,'ova_overall_average')

age            0.52
wage           0.58
power          0.55
shot_power     0.54
vision         0.51
composure      0.70
total_stats    0.61
passing        0.70
dribbling      0.65
physicality    0.55
Name: ova_overall_average, dtype: float64

In [27]:
# (3) We filter only the values that are lowly correlated
filtered_matrix_lowly = corr_matrix_cat[((corr_matrix_cat <= .5) & (corr_matrix_cat >= .0) | (corr_matrix_cat >= -.5) & (corr_matrix_cat <= -.0))]
getcollinearity(filtered_matrix_lowly,'ova_overall_average')

height_cm                   0.02
weight_kg                   0.15
value                       0.37
release_clause              0.15
attacking                   0.43
crossing                    0.39
finishing                   0.30
heading_accuracy            0.30
short_passing               0.49
volleys                     0.36
skill                       0.44
dribbling                   0.35
curve                       0.40
free_kick_accuracy          0.37
long_passing                0.48
ball_control                0.43
movement                    0.31
acceleration                0.15
sprint_speed                0.16
agility                     0.23
balance                     0.10
jumping                     0.26
stamina                     0.34
strength                    0.34
long_shots                  0.39
mentality                   0.49
aggression                  0.38
interceptions               0.30
positioning                 0.34
penalties                   0.31
defending 

In [28]:
# Check the Collinearity between the chosen variables
getcollinearity(filtered_matrix, 'reactions')

ova_overall_average         0.87
age                         0.51
height_cm                  -0.00
weight_kg                   0.12
value                       0.33
wage                        0.52
release_clause              0.14
attacking                   0.44
crossing                    0.37
finishing                   0.33
heading_accuracy            0.33
short_passing               0.49
volleys                     0.38
skill                       0.44
dribbling                   0.35
curve                       0.40
free_kick_accuracy          0.37
long_passing                0.47
ball_control                0.43
movement                    0.33
acceleration                0.13
sprint_speed                0.13
agility                     0.23
reactions                   1.00
balance                     0.13
power                       0.55
shot_power                  0.52
jumping                     0.26
stamina                     0.36
strength                    0.31
long_shots

##### (1) check collinearity within these variables:
(1) reactions, composure, base_stats and passing

- reactions <> base_stats             0.75
- base_stats <> passing               0.85

--> Decision to drop "base_stats", for the first "linear regression" round, because it is colinear to reactions and passing

--> Add "base_stats", for the 2nd "linear regression" round

##### (2) check collinearity within these variables:
(2) reactions, composure, base_stats, passing, age, growth, wage, power, shot_power, vision, total_stats, dribbling, physicality, lcm_left_center_midfielder, cm_center_midfielder, rcm_right_center_midfielder

- reactions <> age                    0.51
- reactions <> wage                   0.52

--> drop power, to many correlations!, --> drop shot_power, to many correlations!, --> drop vision, to many correlations!, --> drop total_stats, to many correlations! 

- reactions <> physicality           0.50
--> let's addd:
- growth

In [29]:
y # from above.  
X = X[['reactions', 'composure', 'passing', 'growth', 'base_stats']]

# 07 - Linear Regression - Train-test-split

In [30]:
# We do the Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [31]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(9104, 5)
(2276, 5)
(9104,)
(2276,)


In [32]:
# We caculate the linear regression based on the "train"-data
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

print("b0 = " , lm.intercept_)
print("b1 = " , lm.coef_)

b0 =  42.30964831103732
b1 =  [25.59024275  5.32337199 -1.19055866 -2.87418305 17.17241722]


In [33]:
# We compare the predicted y-data (applied linear regression on x_train) and compare it with our y_train data.
y_predicted = lm.predict(X_train)
R2p = r2_score(y_train, y_predicted)
print("R2-score is ", R2p) # our r2 score is quite good.

R2-score is  0.8525087922008434


# 07 - Linear Regression - Apply linear regression

In [34]:
# We apply the linear regression on the x-test and compare it with our y-test
y_predicted_test = lm.predict(X_test)
R2 = r2_score(y_test, y_predicted_test)
print("R2-score is ", R2)
# our r2 score is a little smaller than for the predicted set. This is normal.
# since it is more difficult to be close to a small dataset, that was defined by a bigger dataset.

R2-score is  0.8485180304493104


In [35]:
mse=mean_squared_error(y_test,y_predicted_test)
print("mean squared error (MSE) is ", mse)
# --> the mean squared error is the error ^2 so the error seems quite high 

mean squared error (MSE) is  7.024066453484097


In [36]:
rmse = np.sqrt(mean_squared_error(y_test,y_predicted_test))
print("root mean squared error (RMSE) is ", rmse)

# (about 3% when considering the OVA is from 0 to 100)

root mean squared error (RMSE) is  2.6502955407810838


In [37]:
mae = mean_absolute_error(y_test, y_predicted_test)
print("mean absolute error (MAE) is ", mae)

mean absolute error (MAE) is  2.088612816099228


##### correlation with (1) (only highly correlated variables)

- R2-score is  0.8060009338979943
- mean squared error (MSE) is  8.995541424871435
- root mean squared error (RMSE) is  2.9992568120905276
- mean absolute error (MAE) is  2.352901664680519

##### correlation with (2) (only highly and moderately correlated variables)


- R2-score is  0.8485180304493104
- mean squared error (MSE) is  7.024066453484097
- root mean squared error (RMSE) is  2.6502955407810838
- mean absolute error (MAE) is  2.088612816099228

--> We can see that (2) is better than (1)

#### Function to scale and encode the  df

In [38]:
def scale_encode(df):
    
    X_num = df.select_dtypes(include = np.number)
    X_cat= df.select_dtypes(include = object) 
    
    # Mix-max scale for 
    transformer = MinMaxScaler().fit(X_num)
    x_normalized = transformer.transform(X_num)
    X_normalized = pd.DataFrame(x_normalized, columns=X_num.columns)
    
    # # I am now one hot encode X categorical
    encoder = OneHotEncoder(drop='first').fit(X_cat)
    cols = encoder.get_feature_names(input_features=X_cat.columns)
    X_cat_encoded = pd.DataFrame(encoder.transform(X_cat).toarray(),columns=cols)
  
    #concatenate
    df = pd.concat([X_normalized, X_cat_encoded], axis=1)
    
    return df

# 08 - Input New Data to be predicted and validate the Linear Regression Model

In [39]:
fifa_validate = pd.read_csv('./fifa21_validate.csv')
fifa_validate.shape

(1999, 101)

In [40]:
fifa_validate.head()

,ID,Name,Age,Nationality,Club,BP,Position,Team & Contract,Height,Weight,...,CDM,RDM,RWB,LB,LCB,CB,RCB,RB,GK,OVA
0,219461,E. Palmer-Brown,23,United States,FK Austria Wien,CB,CB,"FK Austria Wien Jun 30, 2021 On Loan","6'2""",194lbs,...,61+2,61+2,59+2,61+2,67+2,67+2,67+2,61+2,16+2,67
1,221896,D. Avdijaj,22,Kosovo,Heart of Midlothian,CAM,LM CAM,Heart of Midlothian 2020 ~ 2020,"5'8""",154lbs,...,47+2,47+2,49+2,45+2,38+2,38+2,38+2,45+2,17+2,68
2,247428,D. Ochoa,19,United States,Real Salt Lake,GK,GK,Real Salt Lake 2018 ~ 2020,"6'2""",176lbs,...,18+2,18+2,16+2,16+2,18+2,18+2,18+2,16+2,53+2,54
3,255120,N. Kenneh,16,England,Leeds United,CDM,CB CDM RB,Leeds United 2020 ~ 2022,"6'3""",170lbs,...,54+2,54+2,53+2,53+2,54+2,54+2,54+2,53+2,11+2,55
4,215556,E. Fernandes,24,Switzerland,1. FSV Mainz 05,CDM,CM CDM,1. FSV Mainz 05 2019 ~ 2023,"6'2""",170lbs,...,72+2,72+2,70+2,69+2,68+2,68+2,68+2,69+2,18+2,70


In [41]:
#Apply the function to clean the dataframe
fifa_validate = clean_data(fifa_validate)

In [42]:
fifa_validate.shape

(1940, 65)

In [43]:
get_percentnan(fifa_validate)

,header_name,percent_nulls


In [44]:
#check the datatype
fifa_colandtype(fifa_validate)

,header_name,row 0,type
0,age,23,int64
1,bp,CB,object
2,height_cm,187.96,float64
3,weight_kg,88.0,float64
4,foot,Right,object
5,growth,7,int64
6,value,975000.0,float64
7,wage,5000,int64
8,release_clause,0.0,float64
9,attacking,230,int64


In [45]:
#Apply the function to scale and encode the new dataframe
fifa_scaled = scale_encode(fifa_validate)
fifa_scaled.shape

(1940, 80)

In [46]:
y_validate = fifa_validate['ova_overall_average']
#X_validate = fifa_scaled.drop(['ova_overall_average'], axis=1)
X_validate = fifa_scaled[['reactions', 'composure', 'passing', 'growth', 'base_stats']]

In [47]:
#predict and inspect the results
y_estimated_fifa_validate = lm.predict(X_validate)

In [48]:
r2_score(y_validate, y_estimated_fifa_validate)

0.7768651938360548

In [49]:
mse=mean_squared_error(y_validate,y_estimated_fifa_validate)
print("mean squared error (MSE) is ", mse)
# --> the mean squared error is the error ^2 so the MSE seems quite high

mean squared error (MSE) is  9.987795184266037


In [50]:
rmse = np.sqrt(mean_squared_error(y_validate,y_estimated_fifa_validate))
print("root mean squared error (RMSE) is ", rmse)

root mean squared error (RMSE) is  3.160347320195367


In [51]:
mae = mean_absolute_error(y_validate, y_estimated_fifa_validate)
print("mean absolute error (MAE) is ", mae)

mean absolute error (MAE) is  2.506053648427233


In [52]:
results = pd.concat([fifa_validate,pd.Series(y_estimated_fifa_validate, name='estimate')],axis=1).head()
results = results[['ova_overall_average','estimate']]
results

,ova_overall_average,estimate
0,67.0,62.817138
1,68.0,65.969876
2,54.0,48.484130
3,55.0,53.259655
4,70.0,69.118114


# 09 - Conclusion

The goal was to built a linear regression model with our dataset "fifa21_train" to predict the overall average of a football player, and then to validate the model with the dataset "fifa_validate".
Because of missing time, we concentrated only on pertinent variables to build our model. Mainly considering variables that are highly and medium correlated, with low collinearity inbetween them.
Considering only 5 Variables ('reactions', 'composure', 'passing', 'growth', 'base_stats') we build a model, that validates the new dataframe with a R2 value of 0.77. This is a good bargain, considering the low amount of variables. The MAE is 2.506, which represents around 2.5% error, considering that the OA is a value between 0 - 100.
The next step to better our result, would be to go through the correlation variables again and add some pertinant variables, which may have been missed.